In [5]:
pip install pillow-heif

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install --upgrade opencv-python

Note: you may need to restart the kernel to use updated packages.


In [12]:
import cv2
import os
from PIL import Image
from PIL.ExifTags import TAGS
import datetime
import pillow_heif
import numpy as np

# Function to extract the timestamp from EXIF metadata
def get_exif_timestamp(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if exif_data is not None:
        for tag, value in exif_data.items():
            if TAGS.get(tag) == 'DateTime':
                return value
    return None

# Function to read .HEIC images using pillow-heif
def read_heic_image(image_path):
    # Use pillow to open the HEIC file with pillow-heif's support
    image_data = Image.open(image_path)
    image_np = np.array(image_data)  # Convert PIL image to NumPy array
    return image_np

# Function to filter images based on date and time
def filter_images_by_time(images_dir, start_time="21:00", end_time="22:00"):
    filtered_images = []
    start_time = datetime.datetime.strptime(start_time, "%H:%M").time()
    end_time = datetime.datetime.strptime(end_time, "%H:%M").time()
    
    for filename in os.listdir(images_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.heic')):  # Include HEIC files
            file_path = os.path.join(images_dir, filename)
            timestamp = get_exif_timestamp(file_path)
            
            if timestamp:
                try:
                    # Convert EXIF timestamp to datetime object
                    timestamp = datetime.datetime.strptime(timestamp, "%Y:%m:%d %H:%M:%S")
                    image_time = timestamp.time()
                    
                    # Filter images based on the time range
                    if start_time <= image_time <= end_time:
                        filtered_images.append(file_path)
                except ValueError:
                    pass
    
    return filtered_images

# Function to stitch images
def stitch_images(images_list):
    images = []
    
    # Read images (handling HEIC files)
    for img_path in images_list:
        if img_path.lower().endswith('.heic'):
            img = read_heic_image(img_path)
        else:
            img = cv2.imread(img_path)
        images.append(img)
    
    # Stitch images using OpenCV's Stitcher API
    stitcher = cv2.Stitcher_create()  # Correct API for OpenCV 4.5.1+
    status, stitched_image = stitcher.stitch(images)
    
    if status == cv2.Stitcher_OK:
        return stitched_image
    else:
        print("Error during stitching:", status)
        return None

# Main function to process and stitch images
def main(images_dir):
    # Filter images based on EXIF timestamp
    filtered_images = filter_images_by_time(images_dir)
    
    if len(filtered_images) > 1:
        # Stitch images if there are more than one filtered image
        stitched_image = stitch_images(filtered_images)
        if stitched_image is not None:
            # Save or display the stitched image
            cv2.imwrite("stitched_output.jpg", stitched_image)
            print("Stitched image saved as 'stitched_output.jpg'")
        else:
            print("Failed to stitch images.")
    else:
        print("Not enough images to stitch (must be more than one).")

if __name__ == "__main__":
    # Directory containing images (update this path to your folder)
    images_dir = "C:/Users/user-307E7B3400/Downloads/drive-download-20250117T123348Z-001/"  # Update this path
    main(images_dir)


Stitched image saved as 'stitched_output.jpg'


In [13]:
import os
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\user-307E7B3400


In [10]:
import cv2
import os
from PIL import Image
from PIL.ExifTags import TAGS
import datetime
import pillow_heif
import numpy as np

# Function to extract the timestamp from EXIF metadata
def get_exif_timestamp(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if exif_data is not None:
        for tag, value in exif_data.items():
            if TAGS.get(tag) == 'DateTime':
                return value
    return None

# Function to read .HEIC images using pillow-heif
def read_heic_image(image_path):
    # Use pillow to open the HEIC file with pillow-heif's support
    image_data = Image.open(image_path)
    image_np = np.array(image_data)  # Convert PIL image to NumPy array
    return image_np

# Function to filter images based on date and time
def filter_images_by_time(images_dir, start_time="21:00", end_time="22:00"):
    filtered_images = []
    start_time = datetime.datetime.strptime(start_time, "%H:%M").time()
    end_time = datetime.datetime.strptime(end_time, "%H:%M").time()
    
    for filename in os.listdir(images_dir):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png', '.heic')):  # Include HEIC files
            file_path = os.path.join(images_dir, filename)
            timestamp = get_exif_timestamp(file_path)
            
            if timestamp:
                try:
                    # Convert EXIF timestamp to datetime object
                    timestamp = datetime.datetime.strptime(timestamp, "%Y:%m:%d %H:%M:%S")
                    image_time = timestamp.time()
                    
                    # Filter images based on the time range
                    if start_time <= image_time <= end_time:
                        filtered_images.append(file_path)
                except ValueError:
                    pass
    
    return filtered_images

# Function to create a strip collage (either horizontal or vertical)
def create_strip_collage(images_list, collage_width=500, collage_height=500, orientation='horizontal'):
    images = []
    
    # Read and resize images (handling HEIC files)
    for img_path in images_list:
        if img_path.lower().endswith('.heic'):
            img = read_heic_image(img_path)
        else:
            img = cv2.imread(img_path)
        
        # Resize each image to fit in the collage
        img_resized = cv2.resize(img, (collage_width, collage_height))
        images.append(img_resized)

    # If horizontal, create a single row
    if orientation == 'horizontal':
        collage = np.hstack(images)  # Stack images horizontally
    elif orientation == 'vertical':
        collage = np.vstack(images)  # Stack images vertically
    
    return collage

# Main function to process images and create a strip collage
def main(images_dir):
    # Filter images based on EXIF timestamp
    filtered_images = filter_images_by_time(images_dir)
    
    if len(filtered_images) > 1:
        # Create strip collage if there are enough filtered images
        collage = create_strip_collage(filtered_images, orientation='horizontal')  # Change to 'vertical' if needed
        if collage is not None:
            # Save or display the collage
            cv2.imwrite("collage_output_strip.jpg", collage)
            print("Collage image saved as 'collage_output_strip.jpg'")
            cv2.imshow('Strip Collage', collage)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print("Failed to create collage.")
    else:
        print("Not enough images to create a collage (must be more than one).")

if __name__ == "__main__":
    # Directory containing images (update this path to your folder)
    images_dir = "C:/Users/user-307E7B3400/Downloads/drive-download-20250117T123348Z-001/"  # Update this path
    main(images_dir)

Collage image saved as 'collage_output_strip.jpg'


In [5]:
import os
print("Current working directory:", os.getcwd())


Current working directory: C:\Users\user-307E7B3400
